<a href="https://colab.research.google.com/github/anvy87/AlgoTrading/blob/master/valueMomentum31082025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install beautifulsoup4==4.11.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 kB 4.0 MB/s eta 0:00:00
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.13.5
    Uninstalling beautifulsoup4-4.13.5:
      Successfully uninstalled beautifulsoup4-4.13.5


In [3]:
pip install openpyxl

In [4]:
pip install pandas

In [5]:
import time
import pandas as pd
import logging

In [6]:
def configure_logging():
    # Configure Application Logging
    log_filepath = "ToolsLog.log"
    format = "%(asctime)s: - %(levelname)s - %(message)s"
    logging.basicConfig(
            filename=log_filepath,
            format=format,
            level=logging.INFO,
            datefmt="%Y-%m-%d %H:%M:%S")

In [7]:
def fetch_screener_data(link):
    cache_index = None
    data = pd.DataFrame()
    current_page = 1
    page_limit = 25
    while current_page < page_limit:
        if current_page == 1:
            url=link
        else:
            url = f'{link}?page={current_page}'
        all_tables = pd.read_html(url, flavor='bs4')
        combined_df = pd.concat(all_tables)
        combined_df = combined_df.drop(
            combined_df[combined_df['S.No.'].isnull()].index)
        # print(combined_df)
        # if cache_index == combined_df.iloc[-2]['S.No.']:
        if len(combined_df.index) < 26:
            data = pd.concat([data, combined_df], axis=0)
            break
        # cache_index = combined_df.iloc[-2]['S.No.']
        # print(cache_index)
        data = pd.concat([data, combined_df], axis=0)
        current_page += 1
        time.sleep(1)
    data = data.iloc[0:].drop(data[data['S.No.'] == 'S.No.'].index)
    return data

In [8]:
pd.set_option("display.max_rows", None, "display.max_columns", None)

In [10]:
configure_logging()
logging.info("Tools : Trending Value Screen extract commenced")

In [11]:
# Fetch PE/PB/Dividend Yield Value Ratio
pbv_link = 'https://www.screener.in/screens/2112737/trendvalue_pricebookvalue/'
pbv_df = fetch_screener_data(pbv_link)
pbv_df = pbv_df[['Name','P/E', 'Div Yld  %', 'CMP / BV']]
pbv_df['P/E'] = pd.to_numeric(pbv_df['P/E'], errors='coerce')
pbv_df['Div Yld  %'] = pd.to_numeric(pbv_df['Div Yld  %'], errors='coerce')
pbv_df['CMP / BV'] = pd.to_numeric(pbv_df['CMP / BV'], errors='coerce')
pbv_df['P/E'] = pbv_df['P/E'].fillna(100000)
pbv_df['CMP / BV'] = pbv_df['CMP / BV'].fillna(100000)
pbv_df['Div Yld  %'] = pbv_df['Div Yld  %'].fillna(0)
# pbv_df.to_excel('D:/pbv.xlsx', index=False)
pbv_df.dropna(inplace=True)
pbv_df = pbv_df[pbv_df['P/E'] > 0]
pbv_df = pbv_df[pbv_df['Div Yld  %'] > 0]
merged_df = pbv_df

In [13]:
# Fetch Price to Free Cash Flow
cashflow_link = 'https://www.screener.in/screens/2112756/trendvalue_cashflow/'
cf_df = fetch_screener_data(cashflow_link)
cf_df = cf_df[['Name','CMP / OCF']]
cf_df['CMP / OCF'] = pd.to_numeric(cf_df['CMP / OCF'], errors='coerce')
cf_df['CMP / OCF'] = cf_df['CMP / OCF'].fillna(100000)
# cf_df.to_excel('D:/cashflow.xlsx', index=False)
cf_df.dropna(inplace=True)
cf_df = cf_df[cf_df['CMP / OCF'] > 0]
merged_df = pd.merge(merged_df, cf_df, on='Name', how='inner')
# print("Free Cash Flow data extraction complete")

In [14]:
# Fetch EV to EBITDA
ev_link = 'https://www.screener.in/screens/2112767/trendvalue_ev/'
ev_df = fetch_screener_data(ev_link)
ev_df = ev_df[['Name','EV / EBITDA']]
ev_df['EV / EBITDA'] = pd.to_numeric(ev_df['EV / EBITDA'], errors='coerce')
ev_df['EV / EBITDA'] = ev_df['EV / EBITDA'].fillna(100000)
# ev_df.to_excel('D:/ev.xlsx', index=False)
ev_df.dropna(inplace=True)
ev_df = ev_df[ev_df['EV / EBITDA'] > 0]
merged_df = pd.merge(merged_df, ev_df, on='Name', how='inner')
# print("EV to EBDITA data extraction complete")

In [15]:
# Fetch Price to Sales ratio
sales_link = 'https://www.screener.in/screens/2112772/trendvalue_pricesales/'
sales_df = fetch_screener_data(sales_link)
sales_df = sales_df[['Name','CMP / Sales']]
sales_df['CMP / Sales'] = pd.to_numeric(sales_df['CMP / Sales'], errors='coerce')
sales_df['CMP / Sales'] = sales_df['CMP / Sales'].fillna(100000)
# sales_df.to_excel('D:/sales.xlsx', index=False)
sales_df.dropna(inplace=True)
sales_df = sales_df[sales_df['CMP / Sales'] > 0]
# print("Price to Sales Ratio data extraction complete")

In [16]:
# Fetch Last 6 months return (Momentum)
momentum_link = 'https://www.screener.in/screens/2112742/trendvalue_momentum/'
mo_df = fetch_screener_data(momentum_link)
mo_df = mo_df[['Name','6mth return  %']]
mo_df['6mth return  %'] = pd.to_numeric(mo_df['6mth return  %'], errors='coerce')
mo_df['6mth return  %'] = mo_df['6mth return  %'].fillna(-100000)
# mo_df.to_excel('D:/mo.xlsx', index=False)
mo_df.dropna(inplace=True)
mo_df = mo_df[mo_df['6mth return  %'] > 0]
merged_df = pd.merge(merged_df, mo_df, on='Name', how='inner')
# print("Momentum / 6 month Returns data extraction complete")

In [17]:
# Final Merged dataset
merged_df = pd.merge(merged_df, sales_df, on='Name', how='inner')
merged_df = merged_df.rename(columns={
    'Name': 'Stock',
    'P/E': 'PE',
    'Div Yld  %': 'Div',
    'CMP / BV': 'BV',
    'CMP / OCF': 'Cashflow',
    'EV / EBITDA': 'EV',
    'CMP / Sales': 'Sales',
    '6mth return  %': '6mo Return'
})
merged_df['PE'] = merged_df['PE'].map(lambda x: float(x))
merged_df['Div'] = merged_df['Div'].map(lambda x: float(x))
merged_df['BV'] = merged_df['BV'].map(lambda x: float(x))
merged_df['6mo Return'] = merged_df['6mo Return'].map(lambda x: float(x))
# merged_df['3mo Return'] = merged_df['3mo Return'].map(lambda x: float(x))
merged_df['Cashflow'] = merged_df['Cashflow'].map(lambda x: float(x))
merged_df['EV'] = merged_df['EV'].map(lambda x: float(x))
merged_df['Sales'] = merged_df['Sales'].map(lambda x: float(x))

In [18]:
# Apply Decile for PE
merged_df['PE_Rank'] = merged_df['PE'].rank()
merged_df['PE_Decile'] = pd.qcut(merged_df['PE_Rank'], q=10, labels=False, duplicates='drop') + 1
# Apply Decile for Div
merged_df['Div_Rank'] = merged_df['Div'].rank(ascending=False)
merged_df['Div_Decile'] = pd.qcut(merged_df['Div_Rank'], q=10, labels=False, duplicates='drop') + 1
# Apply Decile for BV
merged_df['BV_Rank'] = merged_df['BV'].rank()
merged_df['BV_Decile'] = pd.qcut(merged_df['BV_Rank'], q=10, labels=False, duplicates='drop') + 1
# Apply Decile for Cashflow
merged_df['Cashflow_Rank'] = merged_df['Cashflow'].rank()
merged_df['Cashflow_Decile'] = pd.qcut(merged_df['Cashflow_Rank'], q=10, labels=False, duplicates='drop') + 1
# Apply Decile for EV
merged_df['EV_Rank'] = merged_df['EV'].rank()
merged_df['EV_Decile'] = pd.qcut(merged_df['EV_Rank'], q=10, labels=False, duplicates='drop') + 1
# Apply Decile for Sales
merged_df['Sales_Rank'] = merged_df['Sales'].rank()
merged_df['Sales_Decile'] = pd.qcut(merged_df['Sales_Rank'], q=10, labels=False, duplicates='drop') + 1

In [19]:
# Consolidated_Rank column
merged_df['Consolidated_Rank'] = merged_df['PE_Decile'] + merged_df['Div_Decile'] + merged_df['BV_Decile'] + merged_df['Cashflow_Decile'] + merged_df['EV_Decile'] + merged_df['Sales_Decile']

In [20]:
# Retain only stocks that has given a postive return in the last 6 months
merged_df = merged_df[merged_df['6mo Return'] > 0]

In [26]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [28]:
# Decile on consolidated rank
merged_df['Consolidated_Decile'] = pd.qcut(merged_df['Consolidated_Rank'], q=10, labels=False, duplicates='drop') + 1
#merged_df = merged_df.sort_values(by=['Consolidated_Decile', '6mo Return'], ascending=[True, False])
merged_df.to_excel('/content/drive/MyDrive/merged.xlsx', index=False)

In [30]:
# Retain only rows in the first decile of Consolidated_Decile
df_final = merged_df[merged_df['Consolidated_Decile'] == 1]
df_final = df_final.sort_values(by=['6mo Return'], ascending=False)
df_final = df_final.head(40)
df_final.to_excel('/content/drive/MyDrive/final.xlsx', index=False)
print("Final Dataset")
print(df_final)
logging.info("Tools : Trending Value Screen extract completed")

Final Dataset
                Stock     PE   Div    BV  Cashflow     EV  6mo Return  Sales  \
62      Hero Motocorp  20.20  3.24  5.28     23.69  13.50       44.75   2.52   
135     Indian Metals  12.99  2.32  1.98      7.95   8.80       39.63   1.83   
158      AGI Greenpac  16.60  0.78  2.75     13.48   8.32       35.97   2.18   
33       Sharda Motor  18.43  1.59  5.43     21.66  11.76       29.91   2.02   
22                KSE   6.87  1.27  2.54      5.28   4.65       22.17   0.46   
137      Welspun Corp  13.62  0.59  2.97     14.77   8.43       18.87   1.54   
102     Nitta Gelatin   9.80  0.73  1.76      9.89   5.48       17.22   1.39   
134  Expleo Solutions  16.14  4.82  2.53      8.97   7.95       15.49   1.56   
122     Nucleus Soft.  16.18  1.23  3.34     18.02  10.71       15.22   3.18   
66   Gulf Oil Lubric.  16.62  3.84  4.21     14.55   9.61       12.17   1.68   
12   Accelya Solution  16.18  3.58  7.52     14.41  10.05       10.11   3.95   
125  Automotive Axles  16.